## Generating SZ Data Example: 

Here we show how to generate a projected map of the Compton-y parameter, extract data around a specified galaxy sample, and produce radial profiles, moment profiles, and thermal energy results.

In [ ]:
import yt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import gridspec
import numpy as np
import caesar
import pandas as pd
import csv
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
from astropy.convolution import convolve, convolve_fft
from Generating_SZ_Data import *
from functions import *

We begin by creating a map of the Compton-y parameter. Here, you need the snapshot, snapshot information, and pixel resolution you want for your analysis.

In [ ]:
filename = '/Volumes/easystore/noagn/snap_m50n512_105.hdf5'
projection_direction = 'x' #x, y, or z
output_name = 'noagn_x_szy.npy' #Name must end in '_x[y,z]_szy.npy' 
z = 0.9927 #redshift of your snapshot
comov =3289.7*1000   #Calculate with https://www.astro.ucla.edu/~wright/CosmoCalc.html
frb=determining_frb_size(50, z, comov, 2.5) #Number of pixels in your fixed resolution buffer, suggested to correspond to resolution at least twice that of your observational comparison

With the snapshot information specified, we run generating_sz_data from Generating_SZ_Data.py, which generates a .npy file containing the 2D map of the Compton-y signal in your snapshot.

In [ ]:
generating_sz_data(filename, projection_direction, output_name, frb, z)

We now need to pull out galaxy sample data from the Caesar file. Below, you can set the path to the Caesar file. You also need to to change the conversion factors in the last three lines of the code below. These convert from Caesar's units (kpccm) to the pixels as set by your frb above. In this example, the box is 50 Mpccm/h: 50/0.68 = 73.5294= 73529.4117647 kpccm. This we divide by 1820 to get our conversion of 40.4 

In [ ]:
gals = caesar.load('/Volumes/easystore/allphys/m50n512_105.hdf5')

#Get galaxy data
galaxy_stellar = [float(i.masses['stellar'].d) for i in gals.galaxies]
halo = [float(i.halo.masses['dm'].d) for i in gals.galaxies]
rad = [float(i.halo.virial_quantities['r200c'].d) for i in gals.galaxies]
ages = [float(i.ages['mass_weighted'].d) for  i in gals.galaxies]
sfr = [float(i.sfr.d )for  i in gals.galaxies]

galaxies_pos = [i.pos for i in gals.galaxies]
xs=[]
ys=[]
zs=[]

conversion_factor = determining_caesar_conversion(50, frb)#conversion needed to align distance scales 

'''Change the conversion scales from Caesar output to sz data'''
for i in range(len(galaxies_pos)):
    xs.append(((galaxies_pos[i][0].value)/conversion_factor).item()) 
    ys.append(((galaxies_pos[i][1].value)/conversion_factor).item())
    zs.append(((galaxies_pos[i][2].value)/conversion_factor).item())

Now we can conduct our analysis. RAFIKI is equipped to do three tasks with the tSZ data: 

1) Radial profiles of the Compton-y parameter

2) Radial profiles of the moments (allowing for measurements of asymmetry)

3) Thermal energy calculation within a given aperture

Below, you can set key parameters to conduct any or all of these analyses. 
The stacking and visualizing portion of the analysis uses these files as is shown in stacking_example.ipynb

In [ ]:
#Name of the SZ data file without '_x[y,z]_szy.npy'
sz_dat_path = 'noagn_'
#Which analysis would you like to run-Radial profiles (rp), Moment profiles (mp) and/or thermal energy (te)?
analysis = 'rpmpte' #currently set to do all three 
#Minimum stellar mass for your sample (ie 1e11):
low_sm = 10**(11) #Will run analysis on all galaxies above this threshold, other population cuts will be made later in the process. 
#Pixel length of box you want to cut around each galaxy:
width = 300
#Standard deviation of Gaussian kernel in pixels for beam convolution: 
beam_kernel = 1.68
#Label assigned to output files:
label = 'example'
#Radius of aperature used to calculate thermal energy
aperature = 50
#If you don't run the first sections of code to generate SZ data, be sure to define the frb here
frb = 1820

Below is the code to generate the data files for data that has been projected in all three directions. If you have only projected in one or two directions, change the specifications of the for loop to reflect that. This process combines all three projections in each output. 

In [ ]:
for i in ["x", "y", "z"]:
    print('Beginning analysis for one projection')
    sz_dat = np.load(sz_dat_path+i+'_szy.npy')
    galaxy_sample = sorting(galaxy_stellar, low_sm) #Gives indices of galaxies in Caesar catalogs larger than low_sm
    with open('galaxy_sample_%s.csv' %label ,'a', newline = '') as f: 
        w = csv.writer(f)
        for j in galaxy_sample:
            c = w.writerow([galaxy_stellar[j],halo[j],rad[j],ages[j],sfr[j]])
    cropping_function = globals()["cropping_sz_"+str(i)]
    stamps = cropping_function(sz_dat, xs, ys, zs, galaxy_sample, width, frb) #Generates stamps around each galaxy
    if 'rp' in analysis:
        make_radial_profiles(stamps, beam_kernel, label)
        print('Completed tSZ radial profiles for one projection')
    if 'mp' in analysis:
        make_moment_profiles(stamps, beam_kernel, label)
        print('Completed moments radial profiles for one projection')
    if 'te' in analysis:
        thermal_energy(stamps, beam_kernel, label, aperature)
        print('Completed thermal energy calculations for one projection')


If you have run all three analyses, you will now have the following files:

**galaxy_sample_[label].csv** - galaxy properties for your sample

**radial_[label].csv** - Radial profile of Compton-y signal around each galaxy

**m_0_[label].csv** - Radial profile of the zeroth moment around each galaxy

**m_1_[label].csv** - Radial profile of the first moment around each galaxy

**m_2_[label].csv** - Radial profile of the second moment around each galaxy

**thermal_energy_[label].csv** - Thermal energy within the radius specified when running around each galaxy

Where [label] refers to the label you assigned above